In [1]:
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import udf

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
df = spark.read.format("csv").option("header",True).option("separator",",").load("hdfs://namenode:9000/user/hadoop/data/air/WHO_AirQuality_Database_2018.csv")

In [4]:
type(df)

pyspark.sql.dataframe.DataFrame

In [5]:
df.printSchema()

root
 |-- ID WHO city: string (nullable = true)
 |-- iso3: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- pm10: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- type_of_stations: string (nullable = true)
 |-- pm10_type: string (nullable = true)
 |-- pm25: string (nullable = true)
 |-- pm25_type: string (nullable = true)
 |-- reference: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- population: string (nullable = true)
 |-- wbinc16_text: string (nullable = true)
 |-- region15: string (nullable = true)
 |-- date_compiled: string (nullable = true)
 |-- population_source: string (nullable = true)
 |-- tempcov_PM10: string (nullable = true)
 |-- tempcov_PM25: string (nullable = true)
 |-- latitude_pop: string (nullable = true)
 |-- longitude_pop: string (nullable = true)
 |-- Region22: string (nullable = true)
 |-- region_abbr: string (nullable = true)
 

In [6]:
df.show(10)

+-----------+----+---------+---------------+----------------+----+--------------------+---------+----------------+---------+--------------------+-----------------+----------------+----------+-------------------+--------+-------------+-----------------+------------+------------+------------+-------------+--------------------+-----------+-----------------+-----------------+---------+----------+---------+----------+
|ID WHO city|iso3|  country|           city|            pm10|Year|    type_of_stations|pm10_type|            pm25|pm25_type|           reference|         latitude|       longitude|population|       wbinc16_text|region15|date_compiled|population_source|tempcov_PM10|tempcov_PM25|latitude_pop|longitude_pop|            Region22|region_abbr|tempcov_PM10_grad|tempcov_PM25_grad|conc_pm25|color_pm25|conc_pm10|color_pm10|
+-----------+----+---------+---------------+----------------+----+--------------------+---------+----------------+---------+--------------------+-----------------+---

In [7]:
df = df.select("iso3", "country", "city", "pm10", "pm25", "Year", "population")
df.show(5)

+----+---------+-------------+----------------+----------------+----+----------+
|iso3|  country|         city|            pm10|            pm25|Year|population|
+----+---------+-------------+----------------+----------------+----+----------+
| ALB|  Albania|       Tirana|     31.61542142|     16.06236628|2013|    453509|
| AUS|Australia|Central Coast|12.8204622829167|             5.5|2014|    297713|
| AUS|Australia|    Devonport|14.9183561110304|             6.4|2013|     29050|
| AUS|Australia|      Geelong|            17.5|7.50752959417486|2014|    173450|
| AUS|Australia|       Hobart|14.2190581683258|             6.1|2013|    170977|
+----+---------+-------------+----------------+----------------+----+----------+
only showing top 5 rows



In [8]:
df = df.withColumn("pm10", df["pm10"].cast(IntegerType()))
df = df.withColumn("pm25", df["pm25"].cast(IntegerType()))
df = df.withColumn("population", df["population"].cast(IntegerType()))

In [9]:
df.printSchema()

root
 |-- iso3: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- pm10: integer (nullable = true)
 |-- pm25: integer (nullable = true)
 |-- Year: string (nullable = true)
 |-- population: integer (nullable = true)



In [10]:
df_toPandas = df.toPandas()
df_toPandas

,iso3,country,city,pm10,pm25,Year,population
0,ALB,Albania,Tirana,31.0,16.0,2013,453509.0
1,AUS,Australia,Central Coast,12.0,5.0,2014,297713.0
2,AUS,Australia,Devonport,14.0,6.0,2013,29050.0
3,AUS,Australia,Geelong,17.0,7.0,2014,173450.0
4,AUS,Australia,Hobart,14.0,6.0,2013,170977.0
...,...,...,...,...,...,...,...
11971,SAU,Saudi Arabia,Riyadh,201.0,51.0,2014,6369710.0
11972,SAU,Saudi Arabia,Riyadh,240.0,69.0,2015,6369710.0
11973,SAU,Saudi Arabia,Riyadh,251.0,73.0,2016,6369710.0
11974,SAU,Saudi Arabia,Dammam,260.0,61.0,2015,1073788.0


In [11]:
df.write.csv("hdfs://namenode:9000/user/hadoop/data/air/air-data.csv")